In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to project
%cd /content/drive/MyDrive/Colab\ Notebooks/italian_teacher

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/italian_teacher


In [2]:
# Install dependencies
!pip install -q transformers trl accelerate peft datasets spacy sentence-transformers bitsandbytes
!python -m spacy download it_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 87.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOConfig, GRPOTrainer
from datasets import Dataset

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA L4


In [4]:
from src.rl.reward_function import ExerciseRewardFunction

# Initialize reward function (loads vocabulary + models)
print("Loading reward function...")
reward_fn = ExerciseRewardFunction()
print("✅ Reward function ready")

Loading reward function...
Loading spaCy model: it_core_news_sm...
✅ spaCy model loaded
Reward function will use device: cuda
Initializing scorers...
Pre-loading CEFR vocabulary (16,887 words)...
✅ Loaded 16887 Italian words from vocabulary list
✅ Loaded vocabulary for all CEFR levels
Loading sentence transformer for topic similarity...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Sentence transformer loaded in cuda
✅ Reward function initialized with all scorers (including coherence)
✅ Reward function ready


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./models/italian_v9_grpo_round2")
model = AutoModelForCausalLM.from_pretrained(
      "./models/italian_v9_grpo_round2",
      device_map="auto",
      load_in_8bit=True)
model.eval()

print(f"Model loaded on device: {next(model.parameters()).device}")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded on device: cuda:0


In [6]:
import os

# Load pre-generated training requests
if os.path.exists("src/rl/validation_requests_1.json"):
    with open("src/rl/validation_requests_1.json", "r") as f:
        validation_requests = json.load(f)
else:
    # If not exists, generate them
    from src.rl.generate_training_requests import generate_training_requests
    print("Generating new training requests...")
    validation_requests = generate_training_requests(
        num_requests=250,
        output_path="src/rl/validation_requests.json"
    )

print(f"✅ Loaded {len(validation_requests)} training requests")

Generating new training requests...
Loading vocabulary...
Generating 250 training requests...
✅ Saved 250 requests to src/rl/validation_requests.json

📊 Request Statistics:
By CEFR level:
  A1: 23
  A2: 67
  B1: 73
  B2: 50
  C1: 23
  C2: 14

Top 5 grammar focuses:
  past_tense: 53
  present_tense: 51
  imperfect_tense: 40
  future_tense: 34
  conditional: 23
✅ Loaded 250 training requests


In [7]:
from src.rl.iterative_training import evaluate_model_on_requests, create_round2_dataset
# After Round 1 training:
low, high = evaluate_model_on_requests(model, tokenizer, reward_fn, validation_requests, batch_size=50, output_path='eval.json')


✅ Model is already quantized and mapped to devices.
✅ SentenceTransformer moved to CUDA for topic scoring.
📊 Evaluating 250 requests in batches of 50 (max_new_tokens=400)
Model device: cuda:0


Evaluating Batches:   0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



✅ Evaluation complete. Processed: 250
  Low scoring (<92): 205
  High scoring (>=92): 45
  Avg score (processed): 85.97
💾 Final results saved to eval.json


In [8]:
type(low)

list